In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install git+https://github.com/okankop/vidaug

In [ ]:
!pip install keras-layer-normalization

In [ ]:
import tensorflow as tf

In [ ]:
from keras.losses import CosineSimilarity

In [ ]:
import tensorflow
from keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from keras.layers import Input, Dense, Flatten, Activation, Dropout, Bidirectional, Permute, multiply
from keras.layers.recurrent import LSTM
from keras.callbacks import CSVLogger
from keras.models import Sequential, Model, load_model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
#from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
import os
import time
import numpy as np
import keras
import tensorflow as tf
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D
from keras.models import Sequential, load_model
from keras.layers import LSTM, Flatten, Bidirectional, Dense, Permute, multiply,Dropout, LayerNormalization,GaussianNoise, GaussianDropout ,GlobalAveragePooling1D,BatchNormalization
from keras_layer_normalization import LayerNormalization
import numpy as np
import h5py
import math
from pandas import read_json
import torch
import tensorflow as tf

In [ ]:
from vidaug import augmentors as va
import cv2
from PIL import Image, ImageSequence
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
from tensorflow import keras
from imutils import paths
import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import imageio
import h5py
from imutils import paths
from tqdm import tqdm
import pandas as pd 
import numpy as np
import shutil
import cv2
import os
from numba import jit
import math
from pandas import read_json

In [ ]:
def load_video(video_path):

  frames = []
  cap = cv2.VideoCapture(video_path)
  n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  

  for frame_idx in range(n_frames-1):
    success, frame = cap.read()
    
    if success:
      if frame_idx % 10 == 0:
        frames.append(frame)


  return frames

In [ ]:
def videoAugmentation(frames,aug_type):
  
  seq = None

  if aug_type == "HorizontalFlip" :
    seq = va.Sequential([ 
      va.HorizontalFlip(), 
    ])

  elif aug_type == "VerticalFlip" :
    seq = va.Sequential([ 
        va.VerticalFlip(),
    ])

  elif aug_type == "InvertColor" :
    seq = va.Sequential([ 
        va.InvertColor(),
    ])
  elif aug_type == "Salt" :
    seq = va.Sequential([ 
        va.Salt(),
    ])
  elif aug_type == "Pepper" :
    seq = va.Sequential([ 
        va.Pepper(),
    ])

  else:
    seq = va.Sequential([ 
        va.Add(),
    ])
  

  video_aug = seq(frames)

  return video_aug

In [ ]:
def build_feature_extractor():
    feature_extractor = tensorflow.keras.applications.VGG19(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(224, 224, 3),
    )
    preprocess_input = keras.applications.vgg19.preprocess_input

    inputs = keras.Input((224, 224, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [ ]:
def extract_feature(feature_extractor, frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224))
    frame = frame[None, ...]
    feature = feature_extractor.predict(frame)
    frame_feature = feature.flatten()

    return feature

In [ ]:
files = glob.glob("/content/drive/MyDrive/summe/videos"+"/*.mp4", recursive = True)
files.sort()
files

In [ ]:
SPLIT = "/content/drive/MyDrive/summe/summe_splits.json"
SPLIT_ID = 0
splits = read_json(SPLIT)
train_keys = splits["train_keys"][0]

In [ ]:
def get_video_paths(train_keys):
  video_paths_for_train = []
  for keys in train_keys:
    video_paths_for_train.append(files[int(keys.split("video_")[1]) - 1])
  return video_paths_for_train
train_path = get_video_paths(train_keys)

In [ ]:
def prepare_videos(video_paths, aug_types ,feature_extractor):
  
  count = 0
  video_feat_for_train = None
  video_features = []
  video_aug = None

  for path in video_paths:
    frames = load_video(path)

    for frame in frames:
      frame_feat = extract_feature(feature_extractor,frame)

      if video_feat_for_train is None:
          video_feat_for_train = frame_feat
      else:
          video_feat_for_train = np.vstack((video_feat_for_train, frame_feat))
    
    video_features.append(video_feat_for_train)

    for aug in aug_types:
      frames_aug = videoAugmentation(frames, aug)

      for frame in frames_aug:
        frame_feat = extract_feature(feature_extractor,frame)

        if video_aug is None:
            video_aug = frame_feat
        else:
            video_aug = np.vstack((video_aug, frame_feat))
      video_features.append(video_aug)
      video_aug = None
    video_feat_for_train = None

  with open('/content/drive/MyDrive/Data_aug_for_sum.npy', 'wb') as f:
    np.save(f, video_features)
  
  return video_features
      
aug_types = ["Salt"]
video_features = prepare_videos(train_path ,aug_types, feature_extractor)

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
class Config:
  BATCH_SIZE = 1
  EPOCHS = 60
  MODEL_PATH = "/content/drive/MyDrive/summe/models/summary_2_salt_model.hdf5"

In [ ]:
def get_model(train_set,reload_model=True):
  
  model = Sequential()
  
  if reload_model:
    if os.path.isfile(Config.MODEL_PATH):
      model = load_model(Config.MODEL_PATH)
      model.fit(train_set,train_set,batch_size=1, epochs=Config.EPOCHS, shuffle=False)
      model.save(Config.MODEL_PATH)
      return model
  
  
  model.add(Bidirectional(LSTM(units=256, return_sequences=True,
                          input_shape=(train_set.shape[1],train_set.shape[2] ))))
  model.add(LayerNormalization())
  model.add(Bidirectional(LSTM(units=128, return_sequences=True )))
  model.add(keras.layers.GaussianNoise(0.2))
  model.add(keras.layers.GaussianDropout(0.2))
  model.add(LayerNormalization())
  model.add(Dense(256,activation="relu", kernel_initializer = tf.keras.initializers.GlorotUniform()))
  model.add(Dropout(0.4))
  model.add(TimeDistributed((Dense(1,activation="sigmoid", kernel_initializer = tf.keras.initializers.GlorotUniform() ,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)))))
  model.compile(loss='cosine similarity', optimizer= tf.keras.optimizers.Nadam(), metrics=["accuracy"])
  model.fit(train_set,train_set,batch_size=Config.BATCH_SIZE, epochs=60, shuffle=False)
  model.save(Config.MODEL_PATH)
  return model

In [ ]:
def attention_block(inputs, time_steps):
    a = Permute((2, 1))(inputs)
    a = Dense(time_steps, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul

In [ ]:
def get_seq_model(train_set):
  frame_features_input = keras.Input((400,512))
  lstm_out = Bidirectional(LSTM(units=256,name="lstm1", return_sequences=True, input_shape=(train_set.shape[1],train_set.shape[2]) ))(frame_features_input)
  lstm_out = Bidirectional(LSTM(units=128, name="lstm2", return_sequences=True ))(lstm_out)

  lstm_out = GaussianNoise(0.2)(lstm_out)
  lstm_out = GaussianDropout(0.1)(lstm_out)
  lstm_out = LayerNormalization()(lstm_out)

  #print(lstm_out.shape)
  #attention_mul = attention_block(lstm_out, lstm_out.shape[1])
  #attention_mul = GlobalAveragePooling1D()(attention_mul)

  x = Dense(256,name="dense1",activation="relu", kernel_initializer = tf.keras.initializers.GlorotUniform() ,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(lstm_out)
  x = Dropout(0.4)(x)
  x = Dense(512,activation="softmax", kernel_initializer = tf.keras.initializers.GlorotUniform() ,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
  
  model = Model(frame_features_input, x)
  model.compile(loss=CosineSimilarity(), optimizer= tf.keras.optimizers.Nadam() , metrics=["accuracy"])
  model.fit(train_set,train_set,batch_size=Config.BATCH_SIZE, epochs=Config.EPOCHS, shuffle=False)
  return model

In [ ]:
def train_all(dataset,size):
  indices = np.arange(len(dataset))
  count = 0
  np.random.shuffle(indices)
  
  data_seq = np.zeros((size,400,512))
  for idx in indices:
    seq = dataset[idx]
    seq_split = np.vsplit(seq,[400])  
    if seq_split[1].shape[0] != 0 :
      seq_split[1]= np.resize(seq_split[1],(400,512))
      data_seq[count] = seq_split[0]
      count += 1
      data_seq[count] = seq_split[1]
      count += 1
      
    else:
      seq_split[0]= np.resize(seq_split[0],(400,512))
      data_seq[count] = seq_split[0]
      count += 1
  model = get_seq_model(data_seq)
  
  return model


model=train_all(video_features,size)